# <span style="font-family:Courier New; color:#CCCCCC">**Dutch Named Entity Recognition CRF**</span>

## <span style="font-family:Courier New; color:#336666">**Load Data and Imports**</span>

In [1]:
from preprocessing import convert_BIO
from ner_evaluation import *
from feature_getter import Feature_getter
import pycrfsuite
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_classification_report

import nltk
nltk.download('conll2002')
from nltk.corpus import conll2002

ned_train = conll2002.iob_sents('ned.train')
ned_val = conll2002.iob_sents('ned.testa')
ned_test = conll2002.iob_sents('ned.testb')

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\jerez\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


## <span style="font-family:Courier New; color:#336666">**Preprocessing Data**</span>

In [2]:
'''names =  []
for sent in ned_train_BIO:
    for token, label in sent:
        if label == 'B-PER':
            names.append(token)
r = Counter(names)
print(r.keys())'''

ned_train_BIO = convert_BIO(ned_train)
ned_val_BIO = convert_BIO(ned_val)
ned_test_BIO = convert_BIO(ned_test)

X_val_BIO = [[word[0] for word in sent] for sent in ned_val_BIO]
y_val_BIO = [[word[1] for word in sent] for sent in ned_val_BIO]
X_test_BIO = [[word[0] for word in sent] for sent in ned_test_BIO]
y_test_BIO = [[word[1] for word in sent] for sent in ned_test_BIO]

## <span style="font-family:Courier New; color:#336666">**Train Classifier**</span>

In [4]:
model = nltk.tag.CRFTagger(feature_func=Feature_getter(language='ned', gazetteers=True))
model.train(ned_train_BIO, 'models/ned_baseline_BIO.tagger')

In [5]:
pred_ned_BIO = model.tag_sents(X_test_BIO)

In [6]:
results, _ = compute_metrics(ned_test_BIO, pred_ned_BIO)
results

{'correct': 3023,
 'incorrect': 541,
 'partial': 25,
 'missed': 337,
 'spurious': 182,
 'possible': 3926,
 'actual': 3771,
 'precision': 0.802,
 'recall': 0.77,
 'F1-score': 0.786}